In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

In [101]:
def transform_input_with_is_missing_token(inputs, targets_present, masked_value):
    """
    e.g. 
        inputs = [a, b, c, d, e]
        targets = [b, c, d, e, f]
        targets_present = [True, 0, True, 0, True]
        masked_value = <m>
        
    then,
        transformed_input = [a, b, <m>, d, <m>]
        
    Args:
        inputs: tensor with shape [batch_size, sequence_length] with tokens
        targets_present: tensor with shape [batch_size, sequence_length] with 1. representing presence of a word of dtype=torch.long
        
    from github.com/tensorflow/models/blob/master/research/maskgan
    """
    #print(targets_present[:,:-1].shape)
    mask = torch.cat((torch.zeros((inputs.size()[0], 1), dtype=torch.long), targets_present[:,:-1]), dim=1)
    print(inputs * (mask))
    inputs[1 - mask] = masked_value
    transformed_input = inputs
    return inputs

In [102]:
torch.from_numpy(np.arange(6))[1 - torch.tensor([1, 0, 1])]

tensor([0, 1, 0], dtype=torch.int32)

In [105]:
# batch_size = 3, length = 4
input_ = torch.from_numpy(np.arange(12).reshape(3, 4)).long()
targets_present = torch.from_numpy(np.array([1, 0, 1, 0]).reshape(1, 4) + np.zeros((3, 1)))
targets_present[1, 1] = 1
targets_present = targets_present.long()
print(input_,'\n', targets_present)
print(transform_input_with_is_missing_token(input_, targets_present, 57))

tensor([[ 0,  1,  2,  3],
        [ 4,  5,  6,  7],
        [ 8,  9, 10, 11]]) 
 tensor([[1, 0, 1, 0],
        [1, 1, 1, 0],
        [1, 0, 1, 0]])
tensor([[ 0,  1,  0,  3],
        [ 0,  5,  6,  7],
        [ 0,  9,  0, 11]])
tensor([[57, 57, 57, 57],
        [57, 57, 57, 57],
        [ 8,  9, 10, 11]])


In [72]:
class GeneratorEncoder(nn.Module): 
    def __init__(self, input_size, hidden_size):
        super(GeneratorEncoder, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size) # based on GRU just for now

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [5]:
class GeneratorDecoder(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(GeneratorDecoder, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)